In [97]:
from concurrent.futures import thread
from ctypes import sizeof
from dataclasses import dataclass
from pickle import TRUE
from tkinter import N

import os
import random
import re

In [98]:
def createDeck():
    numbers = list(range(2,15))
    suits = ['H','S','C','D']
    deck = []
    for i in numbers:
        for s in suits:
            card = str(i)+s
            deck.append(card)
    return deck

In [99]:
def dealMyHand(deck):
    myHand = [deck.pop(), deck.pop(), deck.pop(), deck.pop(), deck.pop(), 0]
    return myHand

In [100]:
def fillInDataMatrix(winPercentage, matrix, numOfHands, myHand):
    handType = occurrenceCounter(myHand, matrix)
    # occurrence percentage
    for i in range(0, len(matrix)):
        matrix[i][1] = ((matrix[i][0]/numOfHands) * 100)
    # [win percentages]
    listOfHandWins = matrix[handType][2]
    listOfHandWins.append(winPercentage)
    # avg win %
    for i in range(0, len(matrix)):
        if len(matrix[i][2]) > 0:
            matrix[i][3] = (int(matrix[i][2][0])/len(matrix[i][2]))

In [101]:
def convertLists(listOfStrings):
    listOfInts = []
    convertibleStr = ''
    for i in range(0, len(listOfStrings)-1):
        convertibleStr += (str(listOfStrings[i]) + " ")
    listOfInts = (re.findall(r'\d+', convertibleStr))
    listOfInts = [int(i) for i in listOfInts]
    return listOfInts

In [102]:
def isolateSuits(handList):
    listOfSuits = []
    convertibleStr = ''
    for i in range(0, len(handList)-1):
        convertibleStr += (handList[i])
    listOfSuits = (re.findall(r'\D', convertibleStr))
    return listOfSuits

In [103]:
def highCardTieBreaker(handList):
    numVals = convertLists(handList)
    numVals = sorted(numVals)
    handList[-1] += (numVals[0]/100000 + numVals[1]/10000 + numVals[2]/1000 + numVals[3]/100 + max(numVals))


In [104]:
def pairCheck(handList):
    onePairConst = 14
    twoPairConst = 30
    numVals = convertLists(handList)
    pairs = []
    for i in numVals:
        if numVals.count(i) == 2:
            pairs.append(i)
            numVals.remove(i)
    notPairs = [x for x in numVals if x not in pairs]
    sorted(notPairs)
    if len(pairs) == 1:
        handList[-1] += (onePairConst + pairs[0] + notPairs[0]/10000 + notPairs[1]/1000 + notPairs[2]/100)
    if len(pairs) == 2:
        numVals = sorted(numVals)
        handList[-1] += (twoPairConst + max(pairs) + min(pairs)/100 + notPairs[0]/1000)

In [105]:
def threeOfAKind(handList):
    threeOAKConst = 45
    numVals = convertLists(handList)
    triple = []
    for i in numVals:
        if numVals.count(i) == 3:
            triple.append(i)
    numVals = set(numVals)
    numVals = list(numVals)
    if triple:
        if triple[0] in numVals:
            numVals.remove(triple[0])
        if len(numVals) == 2:
            handList[-1] += threeOAKConst + triple[0] + min(numVals)/1000 + max(numVals)/100

In [106]:
def straight(handList):
    straightConst = 60
    numVals = convertLists(handList)
    numVals = sorted(numVals)
    span = (numVals[-1] - numVals[0])
    setSize = set(numVals)
    if span == 4 and setSize == 5:
        handList[-1] += (max(numVals) + straightConst)
    elif numVals == [2,3,4,5,14]:
        handList[-1] += (numVals[3] + straightConst)

In [107]:
def flush(handList):
    flushConst = 75
    straightFlushConst = 58
    onlySuits = isolateSuits(handList)
    numVals = convertLists(handList)
    numVals = sorted(numVals)
    span = (numVals[-1] - numVals[0])
    if ((len(set(onlySuits)) == 1) and (span == 4)):
        handList[-1] += (max(numVals) + flushConst + straightFlushConst)
    elif len(set(onlySuits)) == 1 and numVals == [2,3,4,5,14]:
        handList[-1] += (numVals[3] + flushConst + straightFlushConst)
    elif len(set(onlySuits)) == 1:
        handList[-1] += (flushConst + numVals[0]/100000 + numVals[1]/10000 + numVals[2]/1000 + numVals[3]/100 + numVals[4])


In [108]:
def fullHouse(handList):
    fullHouseConst = 90
    numVals = convertLists(handList)
    numValSet = set(numVals)
    if len(numValSet) == 2:
        triple = []
        double = []
        for i in numVals:
            if numVals.count(i) == 3:
                triple.append(i)
        size_of_numVals = len(numVals)
        for i in numVals:
            while i < size_of_numVals:
                if (triple) and (numVals[i] != triple[0]):
                    double.append(i)
                if len(double) == 2:
                    handList[-1] += (fullHouseConst + triple[0] + double[0]/100)


In [109]:
def fourOfAKind(handList):
    fourOfAKindConst = 118
    numVals = convertLists(handList)
    quads = []
    for i in numVals:
        if numVals.count(i) == 4:
            quads.append(i)
            remainingCard = [x for x in numVals if x not in quads]
            handList[-1] += (fourOfAKindConst + quads[0] + remainingCard[0]/100)


In [110]:
def score(currentHand):
    flush(currentHand)
    straight(currentHand)
    fourOfAKind(currentHand)
    fullHouse(currentHand)
    if currentHand[-1] == 0:
        pairCheck(currentHand)
        threeOfAKind(currentHand)
    if currentHand[-1] == 0:
        highCardTieBreaker(currentHand)

In [111]:
def occurrenceCounter(handList, dataMatrix):
    score = handList[-1]
    handType = 0
    if 15 < score < 30:
        handType = 1
    if 30 < score < 45:
        handType = 2
    if 45 < score < 60:
        handType = 3
    if 60 < score < 75:
        handType = 4
    if 75 < score < 89:
        dataMatrix[5][0] += 1
        handType = 5
    if 90 < score < 117:
        handType = 6
    if 118 < score < 133:
        handType = 7
    if 133 < score < 147:
        handType = 8
    if score == 147:
        handType = 9
    dataMatrix[handType][0] += 1
    return handType

In [112]:
def handLabeler(category):
    handLabelStr = ''
    if category == 0:
        handLabelStr = 'High Card'
    elif category == 1:
        handLabelStr = '1 Pair'
    elif category == 2:
        handLabelStr = '2 Pair'
    elif category == 3:
        handLabelStr = '3 of a Kind'
    elif category == 4:
        handLabelStr = 'Straight'
    elif category == 5:
        handLabelStr = 'Flush'
    elif category == 6:
        handLabelStr = 'Full House'
    elif category == 7:
        handLabelStr = '4 of a Kind'
    elif category == 8:
        handLabelStr = 'Straight Flush'
    elif category == 9:
        handLabelStr = 'Royal Flush'
    return handLabelStr

In [113]:
def printCSV(matrix, winPercent):
    csvFile = open('output.csv', 'a')
    csvFile.write('Card 1, Card 2, Card 3, Card 4, Card 5, Score, Win Percentage\n')
    csvFile.write('%s, %s%%  \n' % (matrix[0], winPercent))

In [114]:
def printHandTypesAndPercents(matrix):
    outputFile = open('output.txt','a')
    for i in range(0, len(matrix)):
        handType = handLabeler(i)
        outputFile.write('The occurrence of a %s is %s%%\n' % (handType, matrix[i][1]))
        outputFile.write('The win percentage of %s is %s%%\n' % (handType, matrix[i][3]))

In [115]:
if os.path.isfile('output.txt'):
    os.remove('output.txt')
if os.path.isfile('output.csv'):
    os.remove('output.csv')

    #changing for testing
    numOfMyHands = 10
    
    highCardPercentages = []
    onePairPercentages = []
    twoPairPercentages = []
    threeOAKPercentages = []
    straightPercentages = []
    flushPercentages = []
    fullHousePercentages = []
    fourOAKPercentages = []
    straighFlushPercentages = []
    royalFlushPercentages = []

    # [ occurrence of hand type, percentage of occurrence, [winPercentages], average win percentages ]
    dataMatrix = [
        [0, 0, highCardPercentages, 0],
        [0, 0, onePairPercentages, 0],
        [0, 0, twoPairPercentages, 0],
        [0, 0, threeOAKPercentages, 0],
        [0, 0, straightPercentages, 0],
        [0, 0, flushPercentages, 0],
        [0, 0, fullHousePercentages, 0],
        [0, 0, fourOAKPercentages, 0],
        [0, 0, straighFlushPercentages, 0],
        [0, 0, royalFlushPercentages, 0]
    ]

    for i in range(0, numOfMyHands):
        #changing for testing
        numOfGames = 11
        deck = createDeck()
        random.shuffle(deck)
        myHand = dealMyHand(deck)
        #fullHouseInsteadOf2Pair = ['3C', '3D', '4H', '7H', '7C', 0]
        score(myHand)
        winCounter = 0

        for i in range(0, numOfGames):
            random.shuffle(deck)

            player2 = [deck[-1], deck[-2], deck[-3], deck[-4], deck[-5], 0]
            player3 = [deck[-6], deck[-7], deck[-8], deck[-9], deck[-10], 0]
            player4 = [deck[-11], deck[-12], deck[-13], deck[-14], deck[-15], 0]
            player5 = [deck[-16], deck[-17], deck[-18], deck[-19], deck[-20], 0]
            player6 = [deck[-21], deck[-22], deck[-23], deck[-24], deck[-25], 0]

            handMatrix = [myHand, player2, player3, player4, player5, player6]
            for i in range(1, len(handMatrix)):
                score(handMatrix[i])

            listOfScores = [handMatrix[0][-1], handMatrix[1][-1], handMatrix[2][-1], handMatrix[3][-1], handMatrix[4][-1], handMatrix[5][-1]]
            listOfScores = sorted(listOfScores)
            
            if myHand[-1] == listOfScores[-1] and listOfScores[-1] > listOfScores[-2]:
                winCounter += 1
            if myHand[-1] == listOfScores[-1] and listOfScores[-1] == listOfScores[-2]:
                winCounter += 0.5
        
        winPercentage = (winCounter/numOfGames * 100)

        fillInDataMatrix(winPercentage, dataMatrix, numOfMyHands, myHand)
        printCSV(handMatrix, winPercentage)

    printHandTypesAndPercents(dataMatrix)